在 COMP90024-2019S1-Team7/machine_learning/ 下建立一个叫checkpoints的文件夹, 把模型文件放进去 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
%matplotlib inline

#这个是要测试得图片路径
test_image_path = '/Users/hanxunhuang/Desktop/test/11555928.jpg'

#先显示图片看看
img = mpimg.imread(test_image_path)
plt.imshow(img)


Benchmark一下看看各个model的performance, 取top5和score, 这个score是softmax之后的probability

In [ ]:
from coconut_inference import coconut_inference
import time

def test_time(coconut):
    start = time.time()
    rs = coconut.inference(image_path=test_image_path, num_of_perdict=5)
    end = time.time()
    print('inference took %.4fs' % (end - start))

# NSFW 模型路径
nsfw_mobilenet_path = 'checkpoints/nsfw_mobilenet_sgd.pth_best.pth'
nsfw_resnet50_path = 'checkpoints/nsfw_resnet50_sgd.pth_best.pth'
nsfw_resnet101_sgd_path = 'checkpoints/nsfw_resnet101_sgd.pth_best.pth'
nsfw_resnet101_adabound_path = 'checkpoints/nsfw_resnet101_adabound.pth_best.pth'

# Food179 模型路径
food179_mobilenet_path = 'checkpoints/food179_mobilenet_sgd.pth_best.pth'
food179_resnet50_path = 'checkpoints/food179_resnet50_sgd.pth_best.pth'
food179_resnet101_sgd_path = 'checkpoints/food179_resnet101_sgd.pth_best.pth'
food179_resnet101_adabound_path = 'checkpoints/food179_resnet101_adabound.pth_best.pth'


# 建立Inference
coconut_nsfw_mobilenet = coconut_inference(model_checkpoint_file_path=nsfw_mobilenet_path)
coconut_nsfw_resnet50 = coconut_inference(model_checkpoint_file_path=nsfw_resnet50_path)
coconut_nsfw_resnet101_sgd = coconut_inference(model_checkpoint_file_path=nsfw_resnet101_sgd_path)
coconut_nsfw_resnet101_adabound = coconut_inference(model_checkpoint_file_path=nsfw_resnet101_adabound_path)

coconut_food179_mobilenet = coconut_inference(model_checkpoint_file_path=food179_mobilenet_path)
coconut_food179_resnet50 = coconut_inference(model_checkpoint_file_path=food179_resnet50_path)
coconut_food179_resnet101_sgd = coconut_inference(model_checkpoint_file_path=food179_resnet101_sgd_path)
coconut_food179_resnet101_adabound = coconut_inference(model_checkpoint_file_path=food179_resnet101_adabound_path)


print('=' * 20 + 'NSFW' + '=' * 20)

test_time(coconut_nsfw_mobilenet)
coconut_nsfw_mobilenet.print_model_details()

test_time(coconut_nsfw_resnet50)
coconut_nsfw_resnet50.print_model_details()

test_time(coconut_nsfw_resnet101_sgd)
coconut_nsfw_resnet101_sgd.print_model_details()

test_time(coconut_nsfw_resnet101_adabound)
coconut_nsfw_resnet101_adabound.print_model_details()

print('=' * 20 + 'Food179' + '=' * 20)
test_time(coconut_food179_mobilenet)
coconut_food179_mobilenet.print_model_details()

test_time(coconut_food179_resnet50)
coconut_food179_resnet50.print_model_details()

test_time(coconut_food179_resnet101_sgd)
coconut_food179_resnet101_sgd.print_model_details()

test_time(coconut_food179_resnet101_adabound)
coconut_food179_resnet101_adabound.print_model_details()



选一个模型跑一个测试

In [ ]:
#这个是要测试得图片路径
test_image_path = '/Users/hanxunhuang/Desktop/gHzHahodcfmutut5.jpg'

#先显示图片看看
img = mpimg.imread(test_image_path)
plt.imshow(img)

start = time.time()
rs = coconut_food179_resnet101_sgd.inference(image_path=test_image_path, num_of_perdict=5)
end = time.time()
print('inference took %.4fs' % (end - start))
print(rs)

这个是当前模型处理的所有类别

In [ ]:
print(coconut.model_idx_to_class)

打开一张url图片 处理

In [ ]:
from skimage import io

#这个是要测试得图片URL
test_image_url = 'http://pbs.twimg.com/media/D5PeaD7UcAE4Gca.jpg'
image = io.imread(test_image_url)

#显示图片看看
plt.imshow(image)

start = time.time()
rs = coconut_food179_resnet101_sgd.inference(image_path=test_image_url, num_of_perdict=5, is_url_image=True)
end = time.time()
print('inference took %.4fs' % (end - start))
print(rs)

In [ ]:
import json
 
temp_json_file_path = '/Users/hanxunhuang/Desktop/twitter_data.json'
with open(temp_json_file_path, 'r') as f:
    for line in f:
        line = line[0:len(line)-2]
        try:
            tweet_json = json.loads(line)
            if 'media' in tweet_json['entities']:
                image_url = tweet_json['entities']['media'][0]['media_url']
                food_179_rs = coconut_food179_resnet101_sgd.inference(image_path=image_url, num_of_perdict=5, is_url_image=True)
                nsfw_rs = coconut_nsfw_resnet101_sgd.inference(image_path=image_url, num_of_perdict=5, is_url_image=True)
                print(image_url)
                print(food_179_rs)
                print(nsfw_rs)
                print()
        except Exception as e:
            print(e)
            pass

In [1]:
from PIL import Image
from io import BytesIO

def reformat_Image(image_file_path, is_url_image=False, is_img_data=False):
        if is_url_image:
            response = requests.get(image_file_path)
            image = Image.open(BytesIO(response.content))
        elif is_img_data:
            image = image_file_path
        else:
            image = Image.open(image_file_path, 'r')
        
        if image.mode == 'P':
            print(image.mode)
            new_image = Image.new("RGB", image.size, (0, 0, 0))
            if len(image.split()) > 3:
                new_image.paste(image, mask=image.split()[3])
                image = new_image
            else:
                image = image.convert('RGB')
            print(image.mode)
        elif image.mode != 'RGB':
            image = image.convert('RGB')
        
        image_size = image.size
        width = image_size[0]
        height = image_size[1]
        
#         if(width != height):
#             bigside = width if width > height else height
#             background = Image.new('RGB', (bigside, bigside), (0, 0, 0))
#             offset = (int(round(((bigside - width) / 2), 0)), int(round(((bigside - height) / 2),0)))
#             background.paste(image, offset)
#             img = background
#             # print("%s has been resized to %s" % (ImageFilePath, str(background.size)))
#         else:
        img = image
            # print("%s is already a square, it has not been resized !" % (ImageFilePath))
#         img = img.resize((self.target_image_size, self.target_image_size))
        # print("%s has been resized to %s" % (ImageFilePath, str(img.size)))
        return img
png = '/Users/hanxunhuang/Desktop/download.png'
jpg = '/Users/hanxunhuang/Data/comp90024_p2_nsfw_v3/train/drawings/0aTr7ly.jpg'
img = reformat_Image(image_file_path=png)
img.show()

P
RGB
RGB
